# Advanced RAG with LlamaParse

<a href="https://colab.research.google.com/github/run-llama/llama_parse/blob/main/examples/demo_advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is a complete walkthrough for using LlamaParse with advanced indexing/retrieval techniques in LlamaIndex over the Apple 10K Filing. 

This allows us to ask sophisticated questions that aren't possible with "naive" parsing/indexing techniques with existing models.

Status:
| Last Executed | Version | State      |
|---------------|---------|------------|
| Aug-18-2025   | 0.6.61  | Maintained |

In [ ]:
%pip install llama-cloud-services "llama-index>=0.13.2<0.14.0" "llama-index-embeddings-huggingface>=0.6.0<0.7.0" torchvision "sentence-transformers<5.0"

In [ ]:
!wget "https://s2.q4cdn.com/470004039/files/doc_financials/2021/q4/_10-K-2021-(As-Filed).pdf" -O apple_2021_10k.pdf

Some OpenAI and LlamaParse details

In [ ]:
import os

# API access to llama-cloud
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-..."

# Using OpenAI API for embeddings/llms
os.environ["OPENAI_API_KEY"] = "sk-..."

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")
llm = OpenAI(model="gpt-5-mini")

Settings.llm = llm
Settings.embed_model = embed_model

## Using brand new `LlamaParse` PDF reader for PDF Parsing

We also compare three different retrieval/query engine strategies:
1. Baseline using default parsing from `SimpleDirectoryReader`
2. Using raw markdown text as nodes for building index and apply simple query engine for generating the results;
3. Using markdown + page screenshots to help retrieve the proper nodes.

In [ ]:
from llama_cloud_services import LlamaParse

result = await LlamaParse(
    # The parsing mode
    parse_mode="parse_page_with_agent",
    # The model to use
    model="openai-gpt-4-1-mini",
    # Whether to use high resolution OCR (Slower)
    high_res_ocr=True,
    # Adaptive long table. LlamaParse will try to detect long tables across pages
    adaptive_long_table=True,
    outlined_table_extraction=True,
    output_tables_as_HTML=True,
    # Whether to take a screenshot of the page, needed for screenshot-retrieval
    take_screenshot=True,
).aparse("./apple_2021_10k.pdf")

markdown_nodes = await result.aget_markdown_nodes(split_by_page=True)
screenshot_image_nodes = await result.aget_image_nodes(
    include_screenshot_images=True,
    include_object_images=False,
    image_download_dir="./images",
)

Started parsing the file under job_id f347cb97-dfe2-4677-991a-5ceba6d9fc6a


In [ ]:
from llama_index.core import SimpleDirectoryReader

baseline_documents = SimpleDirectoryReader(
    input_files=["apple_2021_10k.pdf"]
).load_data()

## Setup Baseline Index

For comparison, we setup a naive RAG pipeline with default parsing and standard chunking, indexing, retrieval.

In [ ]:
from llama_index.core import VectorStoreIndex

baseline_index = VectorStoreIndex.from_documents(baseline_documents)
baseline_query_engine = baseline_index.as_query_engine(similarity_top_k=3)

2025-08-18 20:53:51,246 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-18 20:53:52,143 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


## Setup our LlamaParse Indexes

Using both the markdown and screenshot images, we can build two different indexes.

1. An index over just the markdown documents
2. A custom index that uses the markdown + screenshot images to help with response quality.

In [ ]:
from llama_index.core import VectorStoreIndex

markdown_index = VectorStoreIndex(nodes=markdown_nodes)
markdown_query_engine = markdown_index.as_query_engine(similarity_top_k=3)

2025-08-18 20:53:53,070 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [ ]:
from llama_index.core.indices import MultiModalVectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

# could also use other API-based multimodal models like voyageai or jinaai
# Note: this may take quite a while if running on CPU!
image_embed_model = HuggingFaceEmbedding(
    model_name="llamaindex/vdr-2b-multi-v1",
    embed_batch_size=2,
    trust_remote_code=True,
    cache_folder="./hf_cache",
    device="cpu",  # set to "cuda" if you have a GPU or remove to auto-detect
)

multi_modal_index = MultiModalVectorStoreIndex(
    nodes=[*markdown_nodes, *screenshot_image_nodes],
    embed_model=Settings.embed_model,
    image_embed_model=image_embed_model,
    show_progress=True,
)

/Users/loganmarkewich/llama_parse/py/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-18 20:53:55,230 - INFO - Load pretrained SentenceTransformer: llamaindex/vdr-2b-multi-v1
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
2025-08-18 20:54:05,369 - INFO - 2 prompts are loaded, with the keys: ['query', 'text']
Generating image embeddings: 100%|██████████| 82/82 [26:06<00:00, 19.11s/it]


Below, we will create a custom query engine that does a few things
1. Retrieves both image nodes and text nodes
2. Combines them into two lists -- one where images and texts come from the same page, and one where we have texts alone
3. Use a Jinja-based `RichPromptTemplate` to format the retrieved content automatically into a list of multimodal chat messages
4. Send our messages to the LLM and return a result


In [ ]:
from llama_index.core.async_utils import asyncio_run
from llama_index.core.llms import LLM
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.prompts import RichPromptTemplate
from llama_index.core.response import Response
from llama_index.core.schema import NodeWithScore
from llama_index.core import Settings

TEXT_IMAGE_PROMPT_TEMPLATE = RichPromptTemplate(
    """
<context>
Here is some retrieved content from a knowledge base:
{% for image_path, text in images_and_texts %}
<page>
<text>{{ text }}</text>
<image>{{ image_path | image }}</image>
</page>
{% endfor %}
{% for text in texts %}
<page>
<text>{{ text }}</text>
</page>
{% endfor %}
</context>

Using the context, answer the following question:
<query>{{ query_str }}</query>
"""
)


class SimpleMultiModalQueryEngine(CustomQueryEngine):
    def __init__(
        self,
        index: MultiModalVectorStoreIndex,
        image_top_k: int = 4,
        text_top_k: int = 4,
        llm: LLM | None = None,
        **kwargs
    ):
        super().__init__(**kwargs)
        self._retriever = index.as_retriever(
            similarity_top_k=text_top_k, image_similarity_top_k=image_top_k
        )
        self._llm = llm or Settings.llm

    def _match_images_and_texts(
        self, text_results: list[NodeWithScore], image_results: list[NodeWithScore]
    ) -> tuple[list[NodeWithScore], list[NodeWithScore]]:
        # combine results, prioritize images and texts
        # if both an image and matching text was retrieved, that is a strong indicator
        images_and_texts = []
        text_keys = {
            (x.metadata["page_number"], x.metadata["file_name"]): x
            for x in text_results
        }
        for image_result in image_results:
            key = (
                image_result.metadata["page_number"],
                image_result.metadata["file_name"],
            )
            # add matching text to results if available
            if key in text_keys:
                text_result = text_keys[key]
                images_and_texts.append(
                    (image_result.node.image_path, text_result.node.text)
                )

                # remove from list
                text_keys.pop(key)

        # get the remaining texts as a fallback
        texts = [result.node.text for result in text_keys.values()]

        return images_and_texts, texts

    def custom_query(self, query_str: str) -> Response:
        # wrap the async method to avoid code duplication
        # asyncio_run is a slightly safer asyncio.run() call
        return asyncio_run(self.acustom_query(query_str))

    async def acustom_query(self, query_str: str) -> Response:
        text_results = await self._retriever.atext_retrieve(query_str)
        image_results = await self._retriever.atext_to_image_retrieve(query_str)

        images_and_texts, texts = self._match_images_and_texts(
            text_results, image_results
        )
        messages = TEXT_IMAGE_PROMPT_TEMPLATE.format_messages(
            images_and_texts=images_and_texts, texts=texts, query_str=str(query_str)
        )

        response = await self._llm.achat(messages)

        return Response(
            response.message.content, source_nodes=[*text_results, *image_results]
        )

In [ ]:
multimodal_query_engine = SimpleMultiModalQueryEngine(
    index=multi_modal_index,
    image_top_k=3,
    text_top_k=3,
)

## Try out the Query Engines and Compare!

Now with our three query engines assembled, we can compare each approach with a rough "vibes-based" evaluation.

In [ ]:
query = "What were the total fair value of marketable securities in 2020"

response_1 = await baseline_query_engine.aquery(query)
print("\n***********Baseline Query Engine***********")
print(response_1)

response_2 = await markdown_query_engine.aquery(query)
print("\n***********Markdown Query Engine***********")
print(response_2)

response_3 = await multimodal_query_engine.aquery(query)
print("\n***********MultiModal Query Engine***********")
print(response_3)

2025-08-18 21:20:29,006 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-18 21:20:38,721 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



***********Baseline Query Engine***********
The total fair value of marketable securities in 2020 was $153,814 million (approximately $153.8 billion).


2025-08-18 21:20:39,233 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-18 21:20:48,185 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



***********Markdown Query Engine***********
The total fair value was $191,830 million (approximately $191.83 billion).


2025-08-18 21:20:48,515 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-18 21:21:09,275 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



***********MultiModal Query Engine***********
The table shows:

- Total fair value (cash, cash equivalents and marketable securities) in 2020: $191,830 million (≈ $191.83 billion).  
- Total marketable securities (current + non‑current) in 2020: $52,927 + $100,887 = $153,814 million (≈ $153.81 billion).


As we can see, the multimodal and markdown query engines are able to retrieve the correct content, while the default query engine struggles to find the correct total value.

We can also inspect the source nodes, and see the pages that were retrieved. Here is the correct page for the total fair value of marketable securities in 2020:

In [ ]:
response_3.source_nodes[4].node.image_path

'images/page_42.jpg'

Lets try a few more queries to see how the query engines perform.

In [ ]:
query = "What were the effective interest rates of all debt issuances in 2021"

response_1 = await baseline_query_engine.aquery(query)
print("\n***********Baseline Query Engine***********")
print(response_1)

response_2 = await markdown_query_engine.aquery(query)
print("\n***********Markdown Query Engine***********")
print(response_2)

response_3 = await multimodal_query_engine.aquery(query)
print("\n***********MultiModal Query Engine***********")
print(response_3)

2025-08-18 21:35:33,281 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-18 21:35:40,959 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



***********Baseline Query Engine***********
- Second quarter 2021 fixed-rate notes (2026–2061): effective interest rates 0.75%–2.81%
- Fourth quarter 2021 fixed-rate notes (2028–2061): effective interest rates 1.43%–2.86%


2025-08-18 21:35:41,285 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-18 21:35:49,132 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



***********Markdown Query Engine***********
- Floating-rate notes (2022): 0.48% – 0.63%
- Fixed-rate 0.000% – 4.650% notes (2022 – 2060): 0.03% – 4.78%
- Second-quarter 2021 fixed-rate notes (0.700% – 2.800%, 2026 – 2061): 0.75% – 2.81%
- Fourth-quarter 2021 fixed-rate notes (1.400% – 2.850%, 2028 – 2061): 1.43% – 2.86%


2025-08-18 21:35:49,411 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-18 21:36:06,767 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



***********MultiModal Query Engine***********
The effective interest rate ranges reported for the 2021 debt issuances were:

- Floating‑rate notes (2022): 0.48% – 0.63%  
- Fixed‑rate 0.000% – 4.650% notes (2022–2060): 0.03% – 4.78%  
- Q2 2021 fixed‑rate notes (0.700% – 2.800%, maturities 2026–2061): 0.75% – 2.81%  
- Q4 2021 fixed‑rate notes (1.400% – 2.850%, maturities 2028–2061): 1.43% – 2.86%


In [ ]:
query = "federal deferred tax in 2019-2021"

response_1 = await baseline_query_engine.aquery(query)
print("\n***********Baseline Query Engine***********")
print(response_1)

response_2 = await markdown_query_engine.aquery(query)
print("\n***********Markdown Query Engine***********")
print(response_2)

response_3 = await multimodal_query_engine.aquery(query)
print("\n***********MultiModal Query Engine***********")
print(response_3)


***********Baseline Query Engine***********
The federal deferred tax amounts for the years 2019 to 2021 are as follows (in millions):

- **2019**: $(2,939)
- **2020**: $(3,619)
- **2021**: $(7,176)

These figures represent the deferred tax expense for each respective year.

***********Markdown Query Engine***********
As of September 25, 2021, the total deferred tax assets and liabilities for the years 2021 and 2020 are as follows:

**Deferred Tax Assets:**
- 2021: $25,176 million
- 2020: $19,336 million

**Deferred Tax Liabilities:**
- 2021: $7,200 million
- 2020: $10,138 million

**Net Deferred Tax Assets:**
- 2021: $13,073 million
- 2020: $8,157 million

The information for 2019 is not provided in the context.

***********MultiModal Query Engine***********
The federal deferred tax assets and liabilities for the years 2019 to 2021 are as follows:

### Deferred Tax Assets (in millions):
- **2021**: $25,176
- **2020**: $19,336
- **2019**: Not specified in the provided content.

### Def

In [ ]:
query = "current state taxes per year in 2019-2021 (include +/-)"

response_1 = await baseline_query_engine.aquery(query)
print("\n***********Baseline Query Engine***********")
print(response_1)

response_2 = await markdown_query_engine.aquery(query)
print("\n***********Markdown Query Engine***********")
print(response_2)

response_3 = await multimodal_query_engine.aquery(query)
print("\n***********MultiModal Query Engine***********")
print(response_3)

2025-08-18 21:36:07,790 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-18 21:36:14,197 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



***********Baseline Query Engine***********
State current tax (in millions):
- 2019: +$475 million
- 2020: +$455 million
- 2021: +$1,620 million


2025-08-18 21:36:14,584 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-18 21:36:22,084 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



***********Markdown Query Engine***********
2019 — Current state taxes: $475 million (change vs prior year: n/a)  
2020 — Current state taxes: $455 million (change vs 2019: −$20 million)  
2021 — Current state taxes: $1,620 million (change vs 2020: +$1,165 million)


2025-08-18 21:36:22,441 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-18 21:36:33,498 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



***********MultiModal Query Engine***********
The current state tax amounts (in millions) per the Note 5 table are:

- 2019: $475
- 2020: $455  (−$20 vs 2019; −4.2%)
- 2021: $1,620 (+$1,165 vs 2020; +256.0%)

All amounts are in millions of dollars.
